1. Install and Import Dependencies

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

2. Instantiate Model

In [52]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Encode and Calculate Sentiment

In [53]:
tokens = tokenizer.encode('This is AMAZING! but not that good', return_tensors='pt')

In [54]:
result = model(tokens)

In [55]:
result.logits

tensor([[-0.3944,  0.2454,  0.9462,  0.2688, -0.7514]],
       grad_fn=<AddmmBackward0>)

In [56]:
int(torch.argmax(result.logits)) + 1

3

4. Collect Reviews

In [57]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [58]:
reviews

['The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.',
 "Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 'Great atmosphere- very attentive service - amazing food! Everything fresh and super delicious.',
 "Six 

5. Load Reviews into DataFrame and Score

In [59]:
import numpy as np
import pandas as pd

In [60]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [61]:
df.tail()

,review
5,"Great service, lovely location, and really ama..."
6,Great food amazing coffee and tea. Short walk ...
7,Ricotta hot cakes! These were so yummy. I ate ...
8,It was ok. Had coffee with my friends. I'm new...
9,We came for brunch twice in our week-long visi...


In [62]:
df['review'].iloc[0]

'The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.'

In [63]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [64]:
for i, review in enumerate(df['review']):
  print(f'Review {i} sentiment is {sentiment_score(review)} stars')
  df.loc[i, 'sentiment'] = sentiment_score(review)

Review 0 sentiment is 4 stars
Review 1 sentiment is 5 stars
Review 2 sentiment is 5 stars
Review 3 sentiment is 4 stars
Review 4 sentiment is 5 stars
Review 5 sentiment is 5 stars
Review 6 sentiment is 5 stars
Review 7 sentiment is 5 stars
Review 8 sentiment is 3 stars
Review 9 sentiment is 5 stars


In [65]:
df

,review,sentiment
0,The food was delicious. The ricotta pancakes w...,4.0
1,Very cute coffee shop and restaurant. They hav...,5.0
2,Great atmosphere- very attentive service - ama...,5.0
3,Six of us met here for breakfast before our wa...,4.0
4,Great place with delicious food and friendly s...,5.0
5,"Great service, lovely location, and really ama...",5.0
6,Great food amazing coffee and tea. Short walk ...,5.0
7,Ricotta hot cakes! These were so yummy. I ate ...,5.0
8,It was ok. Had coffee with my friends. I'm new...,3.0
9,We came for brunch twice in our week-long visi...,5.0
